<a href="https://colab.research.google.com/github/Xiao-Tu-SD/Skin-Cancer-Classification/blob/main/Haralick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
print(os.path.exists('/content/drive/MyDrive/Skin cancer classification/Train_path/AK/ISIC_0028393.jpg'))


True


Riddhi code for refernces

In [3]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def extract_features(image):
    features = {}

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Entropy (Texture)
    entropy = -np.sum(gray / 255.0 * np.log2(gray / 255.0 + 1e-6))
    features['entropy'] = entropy

    # Haralick Features
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    features['contrast'] = graycoprops(glcm, 'contrast')[0, 0]
    features['correlation'] = graycoprops(glcm, 'correlation')[0, 0]
    features['homogeneity'] = graycoprops(glcm, 'homogeneity')[0, 0]
    features['energy'] = graycoprops(glcm, 'energy')[0, 0]

    # Color Features (RGB/HSV Mean)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    features['mean_hue'] = np.mean(hsv[:, :, 0])
    features['mean_saturation'] = np.mean(hsv[:, :, 1])
    features['mean_value'] = np.mean(hsv[:, :, 2])

    return features

# Example usage
image = cv2.imread('/content/drive/MyDrive/Skin cancer classification/Train_path/AK/ISIC_0028393.jpg')
features = extract_features(image)
print(features)

{'entropy': 115394.38025934197, 'contrast': 43.710517273576095, 'correlation': 0.9191522100666146, 'homogeneity': 0.22269187329107848, 'energy': 0.03195640112844987, 'mean_hue': 7.682066666666667, 'mean_saturation': 81.67766296296297, 'mean_value': 192.62321481481482}


In [4]:
import cv2## OpenCV library for image processing
import numpy as np# Import the NumPy library for numerical operations
from skimage.feature import graycomatrix, graycoprops  # Correct spelling of graycomatrix # Functions for GLCM and feature extraction
from skimage.color import rgb2gray # Converts RGB images to grayscale
from skimage import img_as_ubyte# Converts image to 8-bit unsigned integers
from skimage.io import imread# Reads the image
##Skimage is provides tools for image conversion, GLCM computation, and feature extraction.

In [5]:
def haralick_features(image, distances=[1, 2, 3], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):##extract Haralick features from an image
    """
    Extract enhanced Haralick texture features from an input image.

    Parameters:
        image (array): Input image (grayscale or RGB).
        distances (list): List of pixel pair distances for GLCM computation.
        angles (list): List of angles in radians for GLCM computation.

    Returns:
        features (dict): Dictionary of aggregated Haralick features.
    """
    # Ensure image is grayscale
    ##If the image has 3 channels (RGB), it is converted to grayscale using rgb2gray. This simplifies computations since GLCM requires grayscale input.
    if len(image.shape) == 3:
        image = rgb2gray(image)
        image = img_as_ubyte(image)  # Convert to 8-bit for GLCM computation

    # Compute GLCM
    ##Creates a matrix (glcm) showing how often pixel pairs with specific relationships (distance & angle) appear together.
    glcm = graycomatrix(image, distances=distances, angles=angles, symmetric=True, normed=True)

    # Initialize feature dictionary
    haralick_features = {}

    # Extract standard Haralick features
    feature_names = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']
    for feature_name in feature_names:
        values = graycoprops(glcm, feature_name)
        haralick_features[f'{feature_name}_mean'] = np.mean(values)#Average value across distances and angles.
        haralick_features[f'{feature_name}_std'] = np.std(values)#Variability in the feature.
        haralick_features[f'{feature_name}_range'] = np.ptp(values)# Difference between maximum and minimum values.

    # Add custom features (e.g., entropy)measuring the randomness of the texture.
    glcm_entropy = -np.sum(glcm * np.log2(glcm + (glcm == 0)))# Calculates entropy to measure texture randomness
    haralick_features['entropy'] = glcm_entropy# Stores the calculated entropy in the feature dictionary

    return haralick_features

In [6]:
# Load the specific image
image_path = '/content/drive/MyDrive/Skin cancer classification/Train_path/AK/ISIC_0028393.jpg'
image = imread(image_path)


In [7]:
# Extract Haralick features
features = haralick_features(image)

# Display the features
print("Extracted Haralick Features:")
for key, value in features.items():# Iterates through the dictionary and prints each key-value pair
    print(f"{key}: {value}")# Prints each feature name (key) and its corresponding value

Extracted Haralick Features:
contrast_mean: 15.362718959303606
contrast_std: 7.239888348230214
contrast_range: 20.64488926361238
dissimilarity_mean: 2.5000114394734925
dissimilarity_std: 0.6674100395434444
dissimilarity_range: 1.8811913778744145
homogeneity_mean: 0.36815048506164705
homogeneity_std: 0.06920314670088903
homogeneity_range: 0.2035848434856049
ASM_mean: 0.0018260019397818648
ASM_std: 0.00047574367763117024
ASM_range: 0.0014382290499386188
energy_mean: 0.04237180892037623
energy_std: 0.005534595612778732
energy_range: 0.016513069667904758
correlation_mean: 0.9729328939762057
correlation_std: 0.01278895475964443
correlation_range: 0.03621001232347343
entropy: 116.71893767886218


In [8]:
# Path to the directory containing AK class images
image_dir = "/content/drive/MyDrive/Skin cancer classification/Train_path/AK"